In [2]:
import pandas as pd

In [ ]:
test_excel_path = "pcast_test.xlsx"
test_df = pd.read_excel(test_excel_path)
test_df.head()

In [ ]:
test_df["Date"]

In [33]:
# Mundie, Craig|Cassel, Christine K. (Christine Karen), 1945-|President's Council of Advisors on Science and Technology (PCAST)
exp_lst = test_df["People and Organizations"][1].split("|")
cleaned_lst = []
for exp in exp_lst:
    exp = exp.strip()
    if "(PCAST)" in exp:
        cleaned_lst.append(exp[:-7])
    elif ", " in exp:
        if  "(" in exp and ")" in exp:
            lparen = exp.index("(") + 1
            rparen = exp.index(")")
            comma_idx = exp.index(",")
            cleaned_lst.append(exp[lparen:rparen] + " " + exp[:comma_idx])
        else:
            comma_idx = exp.index(",")
            cleaned_lst.append(exp[comma_idx+1: ]+ " " + exp[:comma_idx])
    else:
        cleaned_lst.append(exp)

In [34]:
cleaned_lst

[' Craig Mundie',
 'Christine Karen Cassel',
 "President's Council of Advisors on Science and Technology "]

In [ ]:
test_df["Description"]

In [ ]:
test_df["Subject"]

In [ ]:
benchmark_excel_path = "/Users/amarkanaka/Desktop/gemini_benchmark.csv"
benchmark_df = pd.read_csv(benchmark_excel_path)
benchmark_df.head()

In [24]:
benchmark_df[" People and Organizations"][0][1:-1].split(",")

["'Executive Office of the President'",
 ' "President\'s Council of Advisors on Science and Technology"',
 " 'John P. Holdren'",
 " 'Office of Science and Technology Policy'",
 " 'Eric Lander'",
 " 'Broad Institute of Harvard and MIT'",
 " 'Harold Varmus'",
 " 'Memorial Sloan-Kettering Cancer Center'",
 " 'Rosina Bierbaum'",
 " 'University of Michigan'",
 " 'Christine Cassel'",
 " 'American Board of Internal Medicine'",
 " 'Christopher Chyba'",
 " 'Princeton University'",
 " 'S. James Gates",
 " Jr.'",
 " 'University of Maryland",
 " College Park'",
 " 'Shirley Ann Jackson'",
 " 'Rensselaer Polytechnic Institute'",
 " 'Richard C. Levin'",
 " 'Yale University'",
 " 'Chad Mirkin'",
 " 'Northwestern University'",
 " 'Mario Molina'",
 " 'University of California",
 " San Diego'",
 " 'Scripps Institution of Oceanography'",
 " 'Ernest J. Moniz'",
 " 'Massachusetts Institute of Technology'",
 " 'Craig Mundie'",
 " 'Microsoft Corporation'",
 " 'Ed Penhoet'",
 " 'Alta Partners'",
 " 'Immune Des

In [39]:
from fuzzywuzzy import fuzz
# people and organizations benchmark
def ppl_orgs_recall_match_v1(gt_entry, bm_list, recall_threshold=0.75, fuzzy_threshold=75):

    # get list of benchmark names
    bm_clean = bm_list
    if bm_clean[0] == "[" :
        bm_clean = bm_clean[1:]
    if bm_clean[-1] == "]":
        bm_clean = bm_clean[:-1]
    bm_clean = bm_clean.split(",")

    """
    get list of ground truth names and orgs, taking out dates, flipping names, and
    removing parentheses at the same time
    """

    gt_entry_lst = gt_entry.split("|")
    cleaned_lst = []
    for exp in gt_entry_lst:
        exp = exp.strip()
        if "(PCAST)" in exp:
            cleaned_lst.append(exp[:-7])
        elif ", " in exp:
            if  "(" in exp and ")" in exp:
                lparen = exp.index("(") + 1
                rparen = exp.index(")")
                comma_idx = exp.index(",")
                cleaned_lst.append(exp[lparen:rparen] + " " + exp[:comma_idx])
            else:
                comma_idx = exp.index(",")
                cleaned_lst.append(exp[comma_idx+1: ]+ " " + exp[:comma_idx])
        else:
            cleaned_lst.append(exp)
    
    match_count = 0

    for gt_entity in cleaned_lst:
        # Check if any benchmark entity matches this ground truth entity
        for bm_entity in bm_clean:
            if fuzz.token_set_ratio(gt_entity, bm_entity) >= fuzzy_threshold:
                print(bm_entity)
                print(gt_entity)
                match_count += 1
                break

    # percentage of ground truth found
    recall = match_count / len(cleaned_lst)
    print(recall)
    return recall >= recall_threshold

ppl_orgs_recall_match_v1(test_df["People and Organizations"][1], benchmark_df[" People and Organizations"][1])


 Craig Mundie
 Craig Mundie
 Christine Cassel
Christine Karen Cassel
President's Council of Advisors on Science and Technology
President's Council of Advisors on Science and Technology 
1.0


True

In [ ]:

def entity_recall_match(gt_list, bm_list, recall_threshold=0.75, fuzzy_threshold=75):
    gt_clean = clean_entities(gt_list)
    bm_clean = clean_entities(bm_list)

    if not gt_clean:
        return False

    match_count = 0
    for gt_entity in gt_clean:
        # Check if any benchmark entity matches this ground truth entity
        for bm_entity in bm_clean:
            if fuzz.token_set_ratio(gt_entity, bm_entity) >= fuzzy_threshold:
                match_count += 1
                break

    # percentage of ground truth found
    recall = match_count / len(gt_clean)
    return recall >= recall_threshold